# Mod 4 Project: Zillow Housing Data Time Series Modeling & Forecasting

Student: Doug Steen

Instructor: James Irving, PhD

Cohort: ds_ft_10072019

Project Review Date: ______________

## Background & Purpose

A fictional real estate investment company has requested assistance with the following question:

- What are the five best zip codes to invest in?

For this study, I am using a Zillow Housing Dataset, obtained from Zillow's research page (https://www.zillow.com/research/data/). The raw dataset is located in this repository ('zillow_data.csv'). This dataset contains monthly average home values for nearly every zip code in the U.S., with most zip codes having data from 1996 - 2018.

### Target Region: Tarrant County, TX

The scope of this analysis is limited to the zip codes in Tarrant County, Texas which have available measurements for the full data range (1996 - 2018). Tarrant County encompasses the western portion of the Dallas-Fort Worth (DFW) Metroplex, and most notably contains the cities of Fort Worth and Arlington.
    
### Method

Modeling and forecasting of home values using ARIMA (Auto-regressive Integrated Moving Average) time series analysis techniques.

### Selection Criteria:

The five 'best' zip codes for investment in Tarrant County will be selected based on the following criteria:
    
-3 year forecasted return on investment (ROI)
    
-Confidence interval of the ROI forecast
    
-Quality of model prediction using a train-test split of Zillow data



## Import Libraries / Packages

In [ ]:
# Import cohort package
#!pip install -U fsds_100719
from fsds_100719.imports import *
import itertools
import statsmodels.api as sm
from tqdm import tqdm_notebook

# Display all columns of large dataframes
pd.set_option('display.max_columns',0)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set default plot style
plt.style.use('seaborn-darkgrid')
%matplotlib inline

## Functions

In [ ]:
def melt_data(df): #formerly called it melt_data_student with REED
   
    melted = pd.melt(df, id_vars=['RegionID','RegionName', 'City', 'State', 'Metro', 'CountyName', 
                                  'SizeRank'], var_name='Month', value_name='MeanValue')
    melted['Month'] = pd.to_datetime(melted['Month'], format='%Y-%m')
    melted = melted.dropna(subset=['MeanValue'])
    
    return melted


def arima_grid_search(df, zip_code, pdq_range = range(0,4), seasonal=False, train_test=True, 
                      train_end='2015-04', show_iters=False):

    # Define the p, d and q parameters to take any value within the specified range
    p = d = q = pdq_range
    
    # Generate all different combinations of p, q and q triplets
    pdq = list(itertools.product(p,d,q))
    
    # Get only time series object for specified zip code
    df_zip = df.loc[df['RegionName'] == zip_code]

    ts_zip = df_zip.drop(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

    ts_zip.set_index('Month', inplace=True)
    
    # If arima model is being fit as part of a train-test split, define correct training interval
    if train_test:
            ts_zip = ts_zip[:train_end]
    
    results = []
    
    if seasonal == False:
        for i in tqdm_notebook(pdq, desc=f'{zip_code} Grid Search Progress'):
            model = sm.tsa.SARIMAX(ts_zip, order=i,enforce_stationarity=False,
                                         enforce_invertibility=False)
            out = model.fit()
            aic = out.aic
            results.append([i, aic])
            if show_iters == True:
                print([i, aic])
        results_df = pd.DataFrame(results, columns=['pdq', 'aic'])
        min_result = results_df.loc[results_df['aic'].idxmin()]
        return zip_code, min_result
    else:
        pdqs = [(x[0], x[1], x[2], 12) for x in pdq]
        for i in tqdm_notebook(pdq, desc=f'{zip_code} Grid Search Loop Progress'):
            for j in tqdm_notebook(pdqs, desc=f'{zip_code} Grid Search Sub-Loop Progress'):
                model = sm.tsa.SARIMAX(ts_zip, order=i, seasonal_order=j, enforce_stationarity=False,
                                         enforce_invertibility=False)
                out = model.fit()
                aic = out.aic
                results.append([i, j, aic])
                if show_iters == True:
                    print([i, j, aic])
        results_df = pd.DataFrame(results, columns=['pdq', 'pdqs', 'aic'])
        min_result = results_df.loc[results_df['aic'].idxmin()]
        return zip_code, min_result
    

def predict_arima(df, zip_code, train_end, test_begin, test_end, pdq, plot=False):
    """Fits an ARIMA model to training data and makes a prediction on testing data for a given zip code. Returns MAPE
    and R^2 of model test predictions.
    
    Parameters
    ----------
    df : pandas dataframe
        Dataframe containing desired time series data for all zip codes
    zip code : int
        Desired zip code for model fit and prediction
    train_end : str
        Date to signify end of training (model fit) portion (Assumed that training data is at beginning of ts)
    test_begin : str
        Date to signify beginning of test portion (Model predictions will begin with this date to end of time series)
    test_end : str
        Date to signify end of test portion (Model predictions will end with this date in time series)
    pdq : tuple
        Tuple of the form (p, d, q) to serve as ARIMA model parameters
    plot : bool
        If True, returns plot of time series and model prediction with 95% conf interval
        (Default = False)
    
    Returns
    -------
    
    """
    import statsmodels.api as sm
    from sklearn.metrics import r2_score
    
    # Get only time series object for specified zip code
    df_zip = df.loc[df['RegionName'] == zip_code]

    ts_zip = df_zip.drop(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

    ts_zip.set_index('Month', inplace=True)
    
    # Define train and test subsets of time series
    ts_zip_train = ts_zip.loc[:train_end]
    ts_zip_test = ts_zip.loc[test_begin:]
    
    mod = sm.tsa.SARIMAX(ts_zip_train, order=pdq, enforce_stationarity=False,
                                 enforce_invertibility=False)

    output = mod.fit()
    
    # Get predictions for test portion of data
    pred_test = output.get_prediction(start=test_begin, end=test_end)
    pred_conf = pred_test.conf_int()
    
    # Model prediction performance on test data
    y_hat_test = pred_test.predicted_mean.values

    y_test = ts_zip_test.values.ravel()
    
    # Creating train-test prediction plot with conf interval if plot is selected
    if plot == True:
        
        plt.rcParams["figure.figsize"] = [8,5]
        ax = ts_zip.plot(color='black')
        pred_test.predicted_mean.plot(ax=ax, label='Test Prediction', alpha=.9, color='red')
        ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='red', alpha=.1, label='95% Conf. Int.')
        ax.set_xlabel('Date')
        ax.set_ylabel('Mean Home Value ($)')
        plt.legend(loc=2)
        plt.title(f'Model Prediction for {zip_code}: {test_begin} - {test_end}')
        plt.show();
        
    # MAPE : Mean Absolute Percentage Error
    mape = round(np.sum(abs((np.subtract(y_test, y_hat_test) / y_test))) / len(y_test), 3)

    # R^2 Score
    r2 = round(r2_score(y_test, y_hat_test), 3)

    model_val_summary = pd.DataFrame(columns = ['Zip Code', 'MAPE (%)', 'R^2'])

    model_val_summary = model_val_summary.append({'Zip Code': str(zip_code), 'pdq': pdq, 
                                                  'MAPE (%)': mape*100, 'R^2': r2}, ignore_index=True)

    return model_val_summary

def forecast_arima(df, zip_code, forecast_begin, forecast_end, pdq, plot=False):
    
    import statsmodels.api as sm
    
     # Get only time series object for specified zip code
    df_zip = df.loc[df['RegionName'] == zip_code]

    ts_zip = df_zip.drop(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

    ts_zip.set_index('Month', inplace=True)
    
    ts_zip.index = pd.DatetimeIndex(ts_zip.index.values,
                               freq='MS')
    
    # Generate forecast arima model object
    fc_model = sm.tsa.SARIMAX(ts_zip, order=pdq, enforce_stationarity=False,
                                 enforce_invertibility=False)
    
    # Fit model
    fc_output = fc_model.fit()
    
    # Obtain model forecast for desired time period, and confidence interval
    forecast = fc_output.get_prediction(start=forecast_begin, end=forecast_end)
    fc_conf = forecast.conf_int()
    
    # Creating forecast plot with conf interval if plot is selected
    if plot == True:
        
        plt.rcParams["figure.figsize"] = [8,5]
        ax = ts_zip.plot(color='black')
        forecast.predicted_mean.plot(ax=ax, label='Forecast Prediction', alpha=.9, color='green')
        ax.fill_between(fc_conf.index,
                fc_conf.iloc[:, 0],
                fc_conf.iloc[:, 1], color='green', alpha=.1, label='95% Conf. Int.')
        ax.set_xlabel('Date')
        ax.set_ylabel('Mean Home Value ($)')
        plt.legend(loc=2)
        plt.title(f'Model Forecast for {zip_code}: {forecast_begin} - {forecast_end}')
        plt.show();
    
    # Calculate 3 year ROI for the forecast
    # Initial value
    init_val = ts_zip.values[-1]

    # Final forecasted value after 3 year forecast
    f_val = forecast.predicted_mean[-1]

    ROI = np.round(((f_val - init_val) / init_val)[0], 3)

    # Calculate lower & upper 95% confidence ROI values

    # lower bound
    l_f_val = fc_conf['lower MeanValue'][-1]

    low_ROI = np.round(((l_f_val - init_val) / init_val)[0], 3)

    # upper bound
    u_f_val = fc_conf['upper MeanValue'][-1]

    high_ROI = np.round(((u_f_val - init_val) / init_val)[0], 3)

    # Size of 95% CL

    size_cl = np.round(high_ROI - low_ROI, 3)

    ROI_summary = pd.DataFrame(columns = ['Zip Code', 'Forecast ROI (%)', 'L 95 ROI (%)', 'H 95 ROI (%)', '95 CL Size (%)'])

    ROI_summary = ROI_summary.append({'Zip Code': str(zip_code), 'Forecast ROI (%)': ROI*100, 'L 95 ROI (%)': low_ROI*100, 
                                      'H 95 ROI (%)': high_ROI*100, '95 CL Size (%)': size_cl*100}, ignore_index=True)

    return ROI_summary

## Obtain / Scrub
-Load Zillow dataset

-Reshape data from Wide Format to Long Format

-Subset dataframe for zip codes in Tarrant County, TX containing data in the full range (1996-2018)

In [ ]:
# Load Zillow data
df = pd.read_csv('zillow_data.csv')

In [ ]:
# Subset data to Tarrant County only, preview Wide Format dataframe
df_tarrant = df.loc[df['CountyName'] == 'Tarrant']
df_tarrant.head()

In [ ]:
# Melt Tarrant Co. dataframe to Long Format
df_melted = melt_data(df_tarrant)
df_melted

In [ ]:
# Check method to later subset dataframe by zip code
# df_melted.loc[df_melted['RegionID'] == 76001]

In [ ]:
# Check for null / missing values
# df_melted.info()

## Explore

In [ ]:
tarrant_zips = df_melted['RegionName'].unique()

In [ ]:
best_tarrant_models = []
for zc in tqdm_notebook(tarrant_zips, desc='Total Grid Search Progress'):
    zip_code, min_result = arima_grid_search(df = df_melted, pdq_range=range(0,7), zip_code=zc)
    best_tarrant_models.append({f'{zip_code}': min_result})
print(best_tarrant_models)

## Model

In [ ]:
# Loop to generate dataframe of all model prediction results, ranked by MAPE (%)

df_all_preds = pd.DataFrame([])
for i in range(len(best_tarrant_models)):
    zip_i = int(list(best_tarrant_models[i].keys())[0])
    pdq_i = list(best_tarrant_models[i].values())[0][0]
    
    summ_i = predict_arima(df=df_melted, zip_code=zip_i, train_end='2015-04', test_begin='2015-05', test_end='2018-04',
                  pdq=pdq_i, plot=False)
    
    df_all_preds = pd.concat([df_all_preds, summ_i], axis=0)
    df_preds_ranked = df_all_preds.sort_values('MAPE (%)')
df_preds_ranked.set_index('Zip Code', inplace=True)
display(df_preds_ranked)
    


## Interpret

In [ ]:
forecast_arima(df=df_melted, zip_code=76133, forecast_begin='2018-05', forecast_end='2021-04', pdq=(0,3,3), plot=True)

In [ ]:
# Loop to generate dataframe of all model forecast results, ranked by ROI (%)

df_all_fcs = pd.DataFrame([])
for i in range(len(best_tarrant_models)):
    zip_i = int(list(best_tarrant_models[i].keys())[0])
    pdq_i = list(best_tarrant_models[i].values())[0][0]
    
    fc_sum_i = forecast_arima(df=df_melted, zip_code=zip_i, forecast_begin='2018-05', forecast_end='2021-04', 
                              pdq=pdq_i, plot=False)
    
    df_all_fcs = pd.concat([df_all_fcs, fc_sum_i], axis=0)
    df_fcs_ranked = df_all_fcs.sort_values('Forecast ROI (%)', ascending=False)
df_fcs_ranked.set_index('Zip Code', inplace=True)
display(df_fcs_ranked)

In [ ]:
df_pred_fc = df_preds_ranked.join(df_fcs_ranked, on='Zip Code')
df_pred_fc['inv_score'] = df_pred_fc['Forecast ROI (%)'] / (df_pred_fc['MAPE (%)'] * df_pred_fc['95 CL Size (%)'])

df_pred_fc.sort_values('inv_score', ascending=False, inplace=True)
top_5_zips = df_pred_fc[:5]
top_5_zips

worst_5_zips = df_pred_fc[-5:]
worst_5_zips

In [ ]:
# Get the arima prediction graphs for the top 5 zip codes
for i, row in top_5_zips.iterrows():
    predict_arima(df=df_melted, zip_code=int(i), train_end='2015-04', test_begin='2015-05', test_end='2018-04', 
                  pdq=row['pdq'], plot=True)

In [ ]:
# Get the 3 year arima forecasts for the top 5 zip codes
for i, row in top_5_zips.iterrows():
    forecast_arima(df=df_melted, zip_code=int(i), forecast_begin='2018-05', forecast_end='2021-04', 
                   pdq=row['pdq'], plot=True)

In [ ]:
# Get the arima prediction graphs for the worst 5 zip codes
for i, row in worst_5_zips.iterrows():
    predict_arima(df=df_melted, zip_code=int(i), train_end='2015-04', test_begin='2015-05', test_end='2018-04', 
                  pdq=row['pdq'], plot=True)

In [ ]:
# Get the 3 year arima forecasts for the top 5 zip codes
for i, row in worst_5_zips.iterrows():
    forecast_arima(df=df_melted, zip_code=int(i), forecast_begin='2018-05', forecast_end='2021-04', 
                   pdq=row['pdq'], plot=True)